<a href="https://colab.research.google.com/github/adnil8130/T5LittleGenQA/blob/main/T5%E7%94%9F%E6%88%90%E5%BC%8F%E9%97%AE%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目标
训练一个生成式问答模型，base模型采用Google T5-Base("uer/t5-base-chinese-cluecorpussmall")

预训练模型地址：https://huggingface.co/uer/t5-base-chinese-cluecorpussmall

模型的评价指标采用BLEU-1，BLEU-2，BLEU-3，BLEU-4。

# 数据集
数据集：链接：https://pan.quark.cn/s/6d4a98cd65f2    

提取码：bzne

数据的格式如下：
```
{"context": "违规分为:一般违规扣分、严重违规扣分、出售假冒商品违规扣分,淘宝网每年12月31日24:00点会对符合条件的扣分做清零处理,详情如下:|温馨提醒:由于出售假冒商品24≤N<48分,当年的24分不清零,所以会存在第一年和第二年的不同计分情况。", "answer": "12月31日24:00", "question": "淘宝扣分什么时候清零", "id": 203}
```

# 1. 准备数据

In [125]:
# ! pip install transformers

In [126]:
import transformers
import numpy as np
import torch
print(transformers.__version__)
print(torch.__version__)
from torch.utils.data import Dataset, random_split
import json

4.49.0
2.5.1+cu124


In [127]:
max_context_len = 0
max_question_len = 0
max_answer_len = 0
context = ""
questiont = ""
answer = ""

save_path_file = '.'
train_data_address = save_path_file + '/train.json'
dev_data_address = save_path_file + '/dev.json'

with open(train_data_address, 'rt', encoding='utf-8') as f:
  for idx, line in enumerate(f):
    sample = json.loads(line.strip())
    if len(sample["context"]) > max_question_len:
        max_context_len = len(sample["context"])
        context = sample["context"]
    if len(sample["question"]) > max_question_len:
        max_question_len = len(sample["question"])
        question = sample["question"]
    if len(sample["answer"]) > max_answer_len:
        max_answer_len = len(sample["answer"])
        answer = sample["answer"]

with open(dev_data_address, 'rt', encoding='utf-8') as f:
  for idx, line in enumerate(f):
    sample = json.loads(line.strip())
    if len(sample["context"]) > max_question_len:
        max_context_len = len(sample["context"])
        context = sample["context"]
    if len(sample["question"]) > max_question_len:
        max_question_len = len(sample["question"])
        question = sample["question"]
    if len(sample["answer"]) > max_answer_len:
        max_answer_len = len(sample["answer"])
        answer = sample["answer"]

print("最长context", max_context_len, context)
print("最长question", max_question_len, question)
print("最长answer", max_answer_len, answer)

最长context 107 2015下半年教师资格证考试时间为11月1日，考生可于2015年10月26日—10月31日登录报名系统，根据提示下载pdf准考证文件。下载后，仔细核对个人信息，并直接打印成准考，按准考证上的要求到指定地点参加考试。
最长question 42 痞子猪身上是什么字母? (问题由猫小逗提供)【答题格式为da+答案,例如答案是爱消除
最长answer 110 如果下雨的时候你拖着行李箱子站在屋檐下面那么其实我没有足够的时间找一个好一点的理由抛弃家里面的狗坐上K667次列车到你在的地方找个商店买一把伞然后给我妹妹弹吉他因为她要参加比赛所以我回不去了我也不会给你说我泡面的碗还没洗


## 构建数据集

In [128]:
train_ratio = 0.9
max_dataset_size = 22000
train_set_size = 20000
valid_set_size = 2000

class GenQA(Dataset):
  def __init__(self, data_file):
    self.data = self.load_data(data_file)

  def load_data(self, data_file):
    Data = {}
    with open(data_file, 'rt', encoding='utf-8') as f:
      for idx, line in enumerate(f):
        if idx >= max_dataset_size:
            break
        sample = json.loads(line.strip())
        Data[idx] = sample
    return Data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

data = GenQA(train_data_address)
data_size = len(data)

train_size = int(train_ratio * data_size)
valid_size = data_size - train_size
train_data, valid_data = random_split(data, [train_size, valid_size])
test_data = GenQA(dev_data_address)

In [129]:
print(f'train set size: {len(train_data)}')
print(f'valid set size: {len(valid_data)}')
print(f'test set size: {len(test_data)}')
print(next(iter(train_data)))

train set size: 13068
valid set size: 1452
test set size: 984
{'context': '无锡哪家 医院做无痛人流比较好啊?有没有无锡的朋友,帮我推荐个医院啊。谢谢了。!|在无锡做人流的价格是多少最近发现自己怀孕... 无锡哪里医院做无痛人流手术比较好老婆怀孕...|我一姐妹在无锡滨湖区蠡溪路203号无锡嘉仕恒信医院做的,已经过去一个月了,到现在还没什么不适应的,当时好像采叫什么薇薇保宫无痛人流术。具体的你可以去看看。', 'answer': '嘉仕恒信医院', 'question': '无锡哪家人流医院好', 'id': 727}


## 数据预处理

In [130]:
from transformers import AutoTokenizer
model_checkpoint = './t5-base-chinese-cluecorpussmall'
# model_checkpoint = 'uer/t5-small-chinese-cluecorpussmall'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,return_token_type_ids=False)

In [131]:
context = train_data[0]["context"]
question = train_data[0]["question"]
answer = train_data[0]["answer"]

inputs = tokenizer(context, question)
targets = tokenizer(answer)

In [132]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['[CLS]', '无', '锡', '哪', '家', '医', '院', '做', '无', '痛', '人', '流', '比', '较', '好', '啊', '?', '有', '没', '有', '无', '锡', '的', '朋', '友', ',', '帮', '我', '推', '荐', '个', '医', '院', '啊', '。', '谢', '谢', '了', '。', '!', '|', '在', '无', '锡', '做', '人', '流', '的', '价', '格', '是', '多', '少', '最', '近', '发', '现', '自', '己', '怀', '孕', '.', '.', '.', '无', '锡', '哪', '里', '医', '院', '做', '无', '痛', '人', '流', '手', '术', '比', '较', '好', '老', '婆', '怀', '孕', '.', '.', '.', '|', '我', '一', '姐', '妹', '在', '无', '锡', '滨', '湖', '区', '蠡', '溪', '路', '203', '号', '无', '锡', '嘉', '仕', '恒', '信', '医', '院', '做', '的', ',', '已', '经', '过', '去', '一', '个', '月', '了', ',', '到', '现', '在', '还', '没', '什', '么', '不', '适', '应', '的', ',', '当', '时', '好', '像', '采', '叫', '什', '么', '薇', '薇', '保', '宫', '无', '痛', '人', '流', '术', '。', '具', '体', '的', '你', '可', '以', '去', '看', '看', '。', '[SEP]', '无', '锡', '哪', '家', '人', '流', '医', '院', '好', '[SEP]']
['[CLS]', '嘉', '仕', '恒', '信', '医', '院', '[SEP]']


In [133]:
import torch

max_input_length = 256
max_target_length = 256
sample_cnt = 4

inputs = [train_data[s_idx]["context"] + train_data[s_idx]["question"] for s_idx in range(sample_cnt)]
targets = [train_data[s_idx]["answer"] for s_idx in range(sample_cnt)]

model_inputs = tokenizer(
    inputs,
    padding=True,
    max_length=max_input_length,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False
)
labels = tokenizer(
    text_target=targets,
    padding=True,
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
    return_token_type_ids=False
)["input_ids"]

end_token_index = torch.where(labels == 102)[-1]
for idx, end_idx in enumerate(end_token_index):
    labels[idx][end_idx+1:] = -100

print('batch_X shape:', {k: v.shape for k, v in model_inputs.items()})
print('batch_y shape:', labels.shape)
print(model_inputs)
print(labels)

batch_X shape: {'input_ids': torch.Size([4, 256]), 'attention_mask': torch.Size([4, 256])}
batch_y shape: torch.Size([4, 8])
{'input_ids': tensor([[ 101, 3187, 7234,  ...,    0,    0,    0],
        [ 101, 1215, 4415,  ...,  511,  738,  102],
        [ 101,  671, 1372,  ...,    0,    0,    0],
        [ 101, 4510, 6228,  ...,  704, 8024,  102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}
tensor([[ 101, 1649,  799, 2608,  928, 1278, 7368,  102],
        [ 101,  679, 7444, 6206,  102, -100, -100, -100],
        [ 101, 5635, 2208, 8933,  702, 6028,  102, -100],
        [ 101, 5494, 1059, 1923,  102, -100, -100, -100]])


In [134]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM
import gc


max_length = 256
train_batch_size = 64
test_batch_size = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

def clean_cuda(device):
    if device == 'cuda':
        # 清理无用变量
        gc.collect()
        torch.cuda.empty_cache()

clean_cuda(device)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model = model.to(device)

def collote_fn(batch_samples):
    batch_inputs, batch_targets = [], []
    for sample in batch_samples:
        batch_inputs.append(sample["context"] + sample["question"])
        batch_targets.append(sample['answer'])
    batch_data = tokenizer(
        batch_inputs,
        text_target=batch_targets,
        padding=True,
        max_length=max_length,
        truncation=True,
        return_tensors="pt",
        return_token_type_ids=False
    )
    batch_data['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(batch_data['labels'])
    end_token_index = torch.where(batch_data['labels'] == 102)[-1]
    for idx, end_idx in enumerate(end_token_index):
        batch_data['labels'][idx][end_idx+1:] = -100
    return batch_data

train_dataloader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

Using cuda device


In [135]:
batch = next(iter(train_dataloader))
print(batch.keys())
print('batch shape:', {k: v.shape for k, v in batch.items()})
print(batch)

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])
batch shape: {'input_ids': torch.Size([64, 256]), 'attention_mask': torch.Size([64, 256]), 'labels': torch.Size([64, 14]), 'decoder_input_ids': torch.Size([64, 14])}
{'input_ids': tensor([[ 101, 4385, 1762,  ...,    0,    0,    0],
        [ 101, 4664, 1164,  ...,    0,    0,    0],
        [ 101,  671,  782,  ...,    0,    0,    0],
        ...,
        [ 101, 3633, 2382,  ...,    0,    0,    0],
        [ 101, 3740, 8020,  ...,    0,    0,    0],
        [ 101,  872, 1962,  ..., 6772, 1962,  102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  101,   727,  5500,  6763,   816,   102,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100],
        [  101,   678,  1286,   127,  4157,  2340,  138

# 2. 模型训练

## 优化模型参数

In [136]:
from tqdm.auto import tqdm
import random

def train_loop(dataloader, model, optimizer, lr_scheduler, epoch, total_loss, sample_ovserve_ratio=0.01):
    progress_bar = tqdm(range(len(dataloader)))
    progress_bar.set_description(f'loss: {0:>7f}')
    finish_batch_num = (epoch-1) * len(dataloader)

    model.train()
    loss_record_step = []
    for batch, batch_data in enumerate(dataloader, start=1):
        batch_data = batch_data.to(device)
        outputs = model(**batch_data)

        random_number = random.uniform(0, 1)
        # if random_number < sample_ovserve_ratio:
        #     print("input:", tokenizer.batch_decode(batch_data["input_ids"].cpu().numpy(), skip_special_tokens=True))


        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        loss_record_step.append(loss.item())
        progress_bar.set_description(f'loss: {total_loss/(finish_batch_num + batch):>7f}')
        progress_bar.update(1)
        del batch_data, outputs, loss
        clean_cuda(device)
    return total_loss, loss_record_step

## 评估指标

In [137]:
# ! pip install sacrebleu

In [138]:
from sacrebleu.metrics import BLEU

predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
bad_predictions_1 = ["This This This This"]
bad_predictions_2 = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]

bleu1 = BLEU(max_ngram_order=1)
print("==========BLEU-1==========")
print(bleu1.corpus_score(predictions, references).score)
print(bleu1.corpus_score(bad_predictions_1, references).score)
print(bleu1.corpus_score(bad_predictions_2, references).score)

bleu2 = BLEU(max_ngram_order=2)
print("==========BLEU-2==========")
print(bleu2.corpus_score(predictions, references).score)
print(bleu2.corpus_score(bad_predictions_1, references).score)
print(bleu2.corpus_score(bad_predictions_2, references).score)

bleu3 = BLEU(max_ngram_order=3)
print("==========BLEU-3==========")
print(bleu3.corpus_score(predictions, references).score)
print(bleu3.corpus_score(bad_predictions_1, references).score)
print(bleu3.corpus_score(bad_predictions_2, references).score)

bleu4 = BLEU(max_ngram_order=4)
print("==========BLEU-4==========")
print(bleu4.corpus_score(predictions, references).score)
print(bleu4.corpus_score(bad_predictions_1, references).score)
print(bleu4.corpus_score(bad_predictions_2, references).score)

==========BLEU-1==========
84.33740467435464
2.634980614046608
0.40867714384640685
==========BLEU-2==========
65.05696445772017
2.1514526621798953
0.40867714384640685
==========BLEU-3==========
53.804523766396294
1.8269935164445736
0.0
==========BLEU-4==========
46.750469682990186
1.683602693167689
0.0


In [139]:
import random

def test_loop(dataloader, model, sample_ovserve_ratio=0.05):
    preds, labels = [], []

    model.eval()
    for batch_data in tqdm(dataloader):
        batch_data = batch_data.to(device)
        with torch.no_grad():
            generated_tokens = model.generate(
                batch_data["input_ids"],
                attention_mask=batch_data["attention_mask"],
                max_length=max_length,
            ).cpu().numpy()
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

        random_number = random.uniform(0, 1)
        if random_number < sample_ovserve_ratio:
            print("input:", tokenizer.batch_decode(batch_data["input_ids"].cpu().numpy(), skip_special_tokens=True))
            print("output:", decoded_preds)

        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)

        del batch_data
        clean_cuda(device)

        preds += [pred.strip() for pred in decoded_preds]
        labels += [[label.strip()] for label in decoded_labels]
    return bleu1.corpus_score(preds, labels).score, bleu2.corpus_score(preds, labels).score, bleu3.corpus_score(preds, labels).score, bleu4.corpus_score(preds, labels).score

## 保存模型

In [140]:
# test_data = GenQA(dev_data_address)
# test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

# test_loop(test_dataloader, model)

In [141]:
import matplotlib.pyplot as plt
import numpy as np

def save_data_and_plot(data, txt_file_path, img_file_path):
    # ===================== 写入文件部分 =====================
    # 追加写入数据（自动创建文件）
    with open(txt_file_path, 'a') as f:
        # 将数字转为字符串并换行写入
        f.write('\n'.join(map(str, data)))
        f.write('\n')  # 添加换行符分隔不同写入批次

    # ===================== 读取文件部分 =====================
    # 从文件读取所有数字
    loaded_data = []
    try:
        with open(txt_file_path, 'r') as f:
            for line in f:
                # 去除空白字符并尝试转换为浮点数
                cleaned_line = line.strip()
                if cleaned_line:
                    loaded_data.append(float(cleaned_line))
    except FileNotFoundError:
        print("错误：文件不存在")
        exit()

    # ===================== 绘图部分 =====================
    plt.figure(figsize=(10, 6))
    plt.plot(loaded_data,
            color='green',
            linestyle='--',
            marker='s',
            markersize=8,
            linewidth=2)

    # 图表装饰
    plt.title("数值变化曲线", fontsize=14, pad=20)
    plt.xlabel("数据索引", fontsize=12, labelpad=10)
    plt.ylabel("测量值", fontsize=12, labelpad=10)
    plt.grid(True, alpha=0.4, linestyle=':')

    # 自动调整坐标轴范围
    plt.xlim(0, len(loaded_data)-1)
    plt.ylim(min(loaded_data)-1, max(loaded_data)+1)

    # 保存和显示
    plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
    plt.close()  # 关闭图表释放内存

    print("操作结果：")
    print(f"- 数据已保存至 {txt_file_path}")
    print(f"- 生成曲线图：{img_file_path}")
    print(f"- 加载数据量：{len(loaded_data)} 条")

In [ ]:
from transformers import AdamW, get_scheduler

learning_rate = 2e-5
epoch_num = 61

optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=epoch_num*len(train_dataloader),
)

total_loss = 0.
best_bleu1 = 0.
best_bleu2 = 0.
best_bleu3 = 0.
best_bleu4 = 0.
best_bleu_weighted_add = 0.
txt_file_path = save_path_file + '/lossdata.txt'
img_file_path = save_path_file + '/lossdata.png'
model_parm_path = None

train_batch_size = 32
test_batch_size = 32

train_dataloader = DataLoader(train_data, batch_size=train_batch_size, shuffle=True, collate_fn=collote_fn)
valid_dataloader = DataLoader(valid_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

if model_parm_path is not None:
    model.load_state_dict(torch.load(model_parm_path))

for t in range(epoch_num):
    clean_cuda(device)
    print(f"Epoch {t+1}/{epoch_num}\n-------------------------------")
    
    # 查看清理后剩余显存
    print(f"可用显存: {torch.cuda.mem_get_info()[0]/1024**3:.2f} GB")
    
    total_loss, loss_record_step = train_loop(train_dataloader, model, optimizer, lr_scheduler, t+1, total_loss)
    save_data_and_plot(loss_record_step, txt_file_path, img_file_path)

    # 查看清理后剩余显存
    print(f"可用显存: {torch.cuda.mem_get_info()[0]/1024**3:.2f} GB")
    valid_bleu1, valid_bleu2, valid_bleu3, valid_bleu4 = test_loop(valid_dataloader, model)
    print(f"BLEU1: {valid_bleu1:>0.2f}\n")
    if valid_bleu1 > best_bleu1:
        best_bleu1 = valid_bleu1
    print(f"BLEU2: {valid_bleu2:>0.2f}\n")
    if valid_bleu2 > best_bleu2:
        best_bleu2 = valid_bleu2
    print(f"BLEU3: {valid_bleu3:>0.2f}\n")
    if valid_bleu3 > best_bleu3:
        best_bleu3 = valid_bleu3
    print(f"BLEU4: {valid_bleu4:>0.2f}\n")
    if valid_bleu4 > best_bleu4:
        best_bleu4 = valid_bleu4

    valid_bleu = 0.1 * valid_bleu1 + 0.2 * valid_bleu2 + 0.3 * valid_bleu3 + 0.4 * valid_bleu4
    if valid_bleu > best_bleu_weighted_add or t % 5 == 0:
        best_bleu_weighted_add = valid_bleu
        print('saving new weights...\n')
        torch.save(model.state_dict(), save_path_file + f'/epoch_{t+1}_loss_{loss_record_step[-1]:0.7f}_valid_bleu_{valid_bleu:0.2f}_model_weights.bin')
print("Done!")

Epoch 1/61
-------------------------------
可用显存: 18.77 GB


  0%|          | 0/409 [00:00<?, ?it/s]

/tmp/ipykernel_1149/1171673335.py:46: UserWarning: Glyph 27979 (\N{CJK UNIFIED IDEOGRAPH-6D4B}) missing from font(s) DejaVu Sans.
  plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
/tmp/ipykernel_1149/1171673335.py:46: UserWarning: Glyph 37327 (\N{CJK UNIFIED IDEOGRAPH-91CF}) missing from font(s) DejaVu Sans.
  plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
/tmp/ipykernel_1149/1171673335.py:46: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from font(s) DejaVu Sans.
  plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
/tmp/ipykernel_1149/1171673335.py:46: UserWarning: Glyph 25968 (\N{CJK UNIFIED IDEOGRAPH-6570}) missing from font(s) DejaVu Sans.
  plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
/tmp/ipykernel_1149/1171673335.py:46: UserWarning: Glyph 21464 (\N{CJK UNIFIED IDEOGRAPH-53D8}) missing from font(s) DejaVu Sans.
  plt.savefig(img_file_path, dpi=300, bbox_inches='tight')
/tmp/ipykernel_1149/1171673335.py:46: UserWarning: Glyp

操作结果：
- 数据已保存至 ./lossdata.txt
- 生成曲线图：./lossdata.png
- 加载数据量：409 条
可用显存: 14.89 GB


  0%|          | 0/46 [00:00<?, ?it/s]

...

  0%|          | 0/409 [00:00<?, ?it/s]

操作结果：
- 数据已保存至 ./lossdata.txt
- 生成曲线图：./lossdata.png
- 加载数据量：5317 条
可用显存: 14.91 GB


  0%|          | 0/46 [00:00<?, ?it/s]

input: ['1 转 不 会 转 的 ， 2 转 到 80 级 ， 转 职 需 要 角 色 到 80 级 ， 而 且 得 到 转 职 材 料 。 转 职 材 料 可 以 去 商 城 买 ， 也 可 以 自 己 打 ， 不 过 比 较 困 难 。 要 蛇 龙 如 意 珠 ×3 玄 武 鳞 片 ×3 沙 漠 石 象 ×3 还 要 转 职 徽 章. 转 职 证 书 获 得 方 法 ： 1 ） 从 商 城 购 买 转 职 合 格 章 ； 2 ） 前 往 艾 利 娅 斯 武 器 店 二 层 ， 寻 找 证 书 制 作 师 ； 如 果 你 是 盾 位 就 转 圣 殿 骑 士 比 较 好 如 果 你 是 游 侠 就 转 火 枪 手 嘛 如 果 你 是 战 士 转 甚 麽 都 可 以 各 有 强 项 如 果 你 是 法 师 我 现 在 觉 嘚 转 吉 它 还 是 不 错 彩 虹 岛 几 级 2 转', '建 议 你 最 好 不 要 服 用 药 物 ， 是 药 三 分 毒 ， 药 物 肯 定 有 一 定 的 副 作 用 。 其 实 比 较 可 靠 的 办 法 还 是 运 动 加 食 补 。 运 动 就 不 说 了 ， 食 补 建 议 多 吃 黑 色 食 物 ， 比 如 ： 黑 芝 麻 、 木 耳 、 海 参 等 ， 同 时 也 推 荐 服 用 澳 纽 琳 玛 卡 ， 这 个 是 主 要 的 ， 建 议 采 纳 哦 。 | 肾 虚 患 者 可 以 吃 狗 肉 ， 韭 菜 进 行 食 补 的 ， 可 以 使 用 六 味 地 黄 丸 进 行 治 疗 的 ， 肾 虚 会 出 现 腰 膝 部 位 酸 痛 或 疼 痛 寒 冷 ， 畏 寒 ， 四 肢 冰 冷 ， 精 神 萎 靡 的 症 状 。 | 六 味 地 黄 丸 以 滋 补 肾 阴 为 主 ， 从 药 方 的 组 成 来 看 ， 它 可 以 达 到 三 阴 同 补 ( 补 肾 阴 、 补 肝 阴 、 补 脾 阴 ) 的 效 果 ， 比 如 熟 地 黄 可 以 补 肾 阴 ; 山 茱 萸 则 是 肝 肾 同 补 ， 通 过 补 肝 来 达 到 补 肾 的 目 的', '国 产 游 戏 其 实 没 几 个 景 气 的 ， 大 都 是 开 时 人 多 ， 过 一 段 时 间 都 变 成 鬼 区 了 ， 回 合 制 游 戏 现 在 真 没 什 么

  0%|          | 0/409 [00:00<?, ?it/s]

In [145]:
test_data = GenQA(dev_data_address)
test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False, collate_fn=collote_fn)

import json

model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.bin'))

model.eval()
with torch.no_grad():
    print('evaluating on test set...')
    sources, preds, labels = [], [], []
    for batch_data in tqdm(test_dataloader):
        batch_data = batch_data.to(device)
        generated_tokens = model.generate(
            batch_data["input_ids"],
            attention_mask=batch_data["attention_mask"],
            max_length=max_length,
        ).cpu().numpy()
        label_tokens = batch_data["labels"].cpu().numpy()

        decoded_sources = tokenizer.batch_decode(
            batch_data["input_ids"].cpu().numpy(),
            skip_special_tokens=True,
            use_source_tokenizer=True
        )
        # print('decoded_sources', decoded_sources)
        
        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        # print('decoded_preds', decoded_preds)
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)
        # print('decoded_labels', decoded_labels)

        sources += [source.strip() for source in decoded_sources]
        preds += [pred.strip() for pred in decoded_preds]
        labels += [[label.strip()] for label in decoded_labels]
    bleu_score = bleu4.corpus_score(preds, labels).score
    print(f"Test BLEU: {bleu_score:>0.2f}\n")
    results = []
    print('saving predicted results...')
    for source, pred, label in zip(sources, preds, labels):
        results.append({
            "sentence": source,
            "prediction": pred,
            "translation": label[0]
        })
    with open('test_data_pred.json', 'wt', encoding='utf-8') as f:
        for exapmle_result in results:
            f.write(json.dumps(exapmle_result, ensure_ascii=False) + '\n')

/tmp/ipykernel_1149/3189609089.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.

evaluating on test set...


  0%|          | 0/16 [00:00<?, ?it/s]

Test BLEU: 0.00

saving predicted results...


In [157]:
model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.bin'))
test_dataloader1 = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collote_fn)
test_loop(test_dataloader1, model, sample_ovserve_ratio=1)


/tmp/ipykernel_1149/442049437.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./epoch_61_loss_4.9312925_valid_bleu_0.00_model_weights.b

  0%|          | 0/984 [00:00<?, ?it/s]

input: ['年 基 准 利 率 4. 35 % 。 从 实 际 看, 贷 款 的 基 本 条 件 是 : 一 是 中 国 大 陆 居 民, 年 龄 在 60 岁 以 下 ; 二 是 有 稳 定 的 住 址 和 工 作 或 经 营 地 点 ; 三 是 有 稳 定 的 收 入 来 源 ; 四 是 无 不 良 信 用 记 录, 贷 款 用 途 不 能 作 为 炒 股, 赌 博 等 行 为 ; 五 是 具 有 完 全 民 事 行 为 能 力 。 2017 年 银 行 贷 款 基 准 利 率']
output: ['']
input: ['年 基 准 利 率 4. 35 % 。 从 实 际 看, 贷 款 的 基 本 条 件 是 : 一 是 中 国 大 陆 居 民, 年 龄 在 60 岁 以 下 ; 二 是 有 稳 定 的 住 址 和 工 作 或 经 营 地 点 ; 三 是 有 稳 定 的 收 入 来 源 ; 四 是 无 不 良 信 用 记 录, 贷 款 用 途 不 能 作 为 炒 股, 赌 博 等 行 为 ; 五 是 具 有 完 全 民 事 行 为 能 力 。 2017 年 银 行 贷 款 基 准 利 率']
output: ['']


KeyboardInterrupt: 